Using Top2Vec to create models for each subreddit
Models will be then loaded to be analyzed with visualization tools

In [1]:
import pandas as pd
from top2vec import Top2Vec
import tqdm as notebook_tqdm

subreddits = ['eczema', 'acne', 'Fungalacne', 'SebDerm', 'SkincareAddiction']

/Users/gunn/repo/submodels/venv/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/gunn/repo/submodels/venv/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/gunn/repo/submodels/venv/lib/python3.10/site-packages/umap/distances.

In [2]:
def zstd_to_df(name):
    comments_filepath = f'originaldata/{name}_comments.zst'
    submissions_filepath = f'originaldata/{name}_submissions.zst'

    comments = pd.read_json(comments_filepath, compression=dict(method='zstd', max_window_size=2147483648), lines=True, nrows=200000)
    submissions = pd.read_json(submissions_filepath, compression=dict(method='zstd', max_window_size=2147483648), lines=True, nrows=200000)

    return comments, submissions

def prepare_df(subreddit):
    data = zstd_to_df(subreddit)
    comments = data[0]
    submissions = data[1]

    sorted_comments = comments.sort_values(by=['score'], ascending=False)
    sorted_submissions = submissions.sort_values(by=['score'], ascending=False)

    # drop row if score is less than 3
    sorted_comments = sorted_comments[sorted_comments['score'] >= 3]
    sorted_submissions = sorted_submissions[sorted_submissions['score'] >= 3]

    # extract body from comments
    comments_body = sorted_comments['body'].to_list()

    # extract title and selftext from submissions
    submissions_title = sorted_submissions['title'].to_list()
    submissions_selftext = sorted_submissions['selftext'].to_list()

    # combine title and selftext
    for each in submissions_selftext:
        submissions_title.append(each)
    
    return comments_body, submissions_title

In [3]:
def create_models(subreddit, comments, submissions):
    commentsmodel = Top2Vec(documents=comments, speed="deep-learn", ngram_vocab=True, workers=8)
    commentsmodel.save(f"{subreddit}_comments_model")
    print(f'{subreddit} comments model saved')
    submodel = Top2Vec(documents=submissions, speed="deep-learn", ngram_vocab=True, workers=8)
    submodel.save(f"{subreddit}_submissions_model")
    print(f'{subreddit} submissions model saved')

In [ ]:
for subreddit in subreddits:
    comments, submissions = prepare_df(subreddit)
    print(f'{subreddit} data prepared')
    create_models(subreddit, comments, submissions)